In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("拍牌数据.xlsx")
data.head()

,拍卖时间,投放数量,最低成交价,平均成交价,最低成交价截止时间,位数,投标人数,中签率
0,45566,4147,93800,93894,11:29:59,第231位,42157,0.098370
1,45536,4260,93100,93255,11:29:56,第55位,41171,0.103471
2,45505,4248,93900,93959,11:29:57,第1705位,41036,0.103519
3,45474,5074,93600,93716,11:29:57,第129位,38145,0.133019
4,45444,6271,93700,93747,11:29:59,第3267位,40720,0.154003


In [17]:
df_all = data.copy()

df_all['date_column'] = pd.to_datetime(df_all['拍卖时间'].astype(int),  unit='D', origin=pd.Timestamp('1899-12-30'))
df_all['y_m'] = df_all['date_column'].dt.strftime('%Y-%m')


target_time_td = pd.to_timedelta('11:30:00')
df_all['seconds_diff'] = (+ target_time_td - pd.to_timedelta(df_all['最低成交价截止时间'].astype(str))).dt.total_seconds()

df_all.head()


# 环比情况

# 计算前一个月的值
df_all['Previous_投标人数'] = df_all['投标人数'].shift(-1)
df_all['Previous_中签率'] = df_all['中签率'].shift(-1)
df_all['Previous_最低成交价'] = df_all['最低成交价'].shift(-1)
df_all['Previous_平均成交价'] = df_all['平均成交价'].shift(-1)


df_all['投标人数_MoM'] = (df_all['投标人数'] - df_all['Previous_投标人数']) / df_all['Previous_投标人数'] 
df_all['中签率_MoM'] = (df_all['中签率'] - df_all['Previous_中签率']) / df_all['Previous_中签率']
df_all['最低成交价_MoM'] = (df_all['最低成交价'] - df_all['Previous_最低成交价']) / df_all['Previous_最低成交价']
df_all['平均成交价_MoM'] = (df_all['平均成交价'] - df_all['Previous_平均成交价']) / df_all['Previous_平均成交价']

# 去掉第一个值为空的行
df_MoM = df_all.dropna(subset=['投标人数_MoM'])


In [4]:
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [6]:
fig = go.Figure()

df_2014 = df_all.copy()[df_all['y_m']>='2013'].reset_index(drop=True)
fig.add_trace(go.Scatter(x=df_2014['y_m'], y=df_2014['最低成交价'], name='最低成交价',
                         fill=None, mode='lines+markers'))
max_a = df_2014['最低成交价'].max()

# fig.add_trace(go.Scatter(x=df_2014['y_m'], y=df_2014['平均成交价'], name='平均成交价',
#                          fill='tonexty', mode='lines+markers'))  # 'tonexty' 填充到下一个 y 值
# max_a = df_2014['平均成交价'].max()


# 添加水平线
fig.add_shape(
    type="line",
    x0=df_2014['y_m'].min(), x1=df_2014['y_m'].max(),
    y0=max_a, y1=max_a,
    line=dict(
        color="gray",
        width=2,
        dash="dash",
    )
)

fig.update_layout(title_text='平均成交价')



In [23]:
df = df_all[df_all['y_m']>='2021'].reset_index(drop=True)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# 添加投标人数
#fig.add_trace(go.Scatter(x=df['y_m'], y=df['投标人数'], name='投标人数',
#                         line=dict(color='red'), mode='lines+markers'),
#              secondary_y=False)
fig.add_trace(go.Bar(x=df['y_m'], y=df['投标人数'], name='投标人数',
                    ),
              secondary_y=False)

# 添加中签率
fig.add_trace(go.Scatter(x=df['y_m'], y=df['中签率'], name='中签率',
                         line=dict(color='red'), mode='lines+markers'),
              secondary_y=True) # purple
# df_all['最低成交价_MoM']
fig.add_trace(go.Scatter(x=df['y_m'], y=df['平均成交价_MoM'], name='平均成交价_MoM',
                         line=dict(color='purple'), mode='lines+markers'),
              secondary_y=True) # purple
# 添加水平线
#min_a = 0 # df['中签率'].min()
#fig.add_shape(
#    type="line",
#    x0=df['y_m'].min(), x1=df['y_m'].max(),
#    y0=min_a, y1=min_a,
#    line=dict(
#        color="gray",
#        width=2,
#        dash="dash",
#    ),
#    secondary_y=True
#)

fig.update_layout(title_text='投标人数与中签率变化趋势')

fig.update_xaxes(title_text='Month')

fig.update_yaxes(title_text='投标人数', secondary_y=False)
fig.update_yaxes(title_text='中签率 (%)', secondary_y=True)

fig.show()


In [24]:
df_MoM.head()

,拍卖时间,投放数量,最低成交价,平均成交价,最低成交价截止时间,位数,投标人数,中签率,date_column,y_m,seconds_diff,Previous_投标人数,Previous_中签率,投标人数_MoM,中签率_MoM
1,45413,7098,93000,93029,11:29:58,第1139位,43374,0.163646,2024-05-01,2024-05,2.0,40720.0,0.154003,0.065177,0.062619
2,45383,8515,92900,93022,11:29:58,第194位,45071,0.188924,2024-04-01,2024-04,2.0,43374.0,0.163646,0.039125,0.154465
3,45352,9595,93400,93497,11:29:57,第3063位,49498,0.193846,2024-03-01,2024-03,3.0,45071.0,0.188924,0.098223,0.026053
4,45323,11804,92700,92842,11:29:47,第1947位,50962,0.231624,2024-02-01,2024-02,13.0,49498.0,0.193846,0.029577,0.194883
5,45292,12483,92900,92990,11:29:57,第1957位,59991,0.208081,2024-01-01,2024-01,3.0,50962.0,0.231624,0.177171,-0.101641


In [8]:



df = df_MoM[df_MoM['y_m']>='2013'].reset_index(drop=True)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# 添加投标人数
#fig.add_trace(go.Scatter(x=df['y_m'], y=df['投标人数_MoM'], name='投标人数_MoM',
#                         line=dict(color='red'), mode='lines+markers'),
#              secondary_y=False)

# 最低成交价_MoM
fig.add_trace(go.Scatter(x=df['y_m'], y=df['最低成交价_MoM'], name='最低成交价_MoM',
                         line=dict(color='red'), mode='lines+markers'),
              secondary_y=False)

# 添加中签率
fig.add_trace(go.Scatter(x=df['y_m'], y=df['中签率_MoM'], name='中签率_MoM',
                         line=dict(color='purple'), mode='lines+markers'),
              secondary_y=True)

fig.update_layout(title_text='投标人数与中签率环比变化')

fig.update_xaxes(title_text='时间')

fig.update_yaxes(title_text='投标人数_MoM', secondary_y=False)
fig.update_yaxes(title_text='中签率_MoM', secondary_y=True)

fig.show()


In [10]:
import plotly.graph_objects as go

# 筛选 seconds_diff 小于或等于 10 的行
filtered_df = df[df['seconds_diff'] <= 10]

fig = go.Figure()

# 使用筛选后的 DataFrame 绘制 seconds_diff
fig.add_trace(go.Scatter(x=filtered_df['y_m'], y=filtered_df['seconds_diff'], name='seconds_diff',
                         line=dict(color='orange'), mode='lines+markers'))

fig.update_layout(title='Seconds_diff 变化趋势（限制在10秒内）',
                  xaxis_title='时间',
                  yaxis_title='Seconds Difference',
                  font=dict(family="Arial, sans-serif", size=12, color="RebeccaPurple"))

fig.show()
